In [1]:
import sys
sys.path.append('/home/jovyan/workspace')

In [ ]:
%%sh
rm -rf '../data/test'

In [2]:
outputPath = '../data/test'

In [ ]:
# from planetengine.utilities import Grouper
# from planetengine.utilities import hashstamp
# from planetengine.wordhash import wordhash as wordhashFn
# from planetengine.checkpoint import Checkpointer
# from planetengine.frame import _scripts_and_stamps
# import underworld as uw

# class Observer:

#     figs = []
#     collectors = []

#     def __init__(
#             self,
#             system,
#             initials,
#             scriptlist,
#             **kwargs
#             ):

#         obsInp = kwargs

#         scripts = {}
#         for index, script in enumerate(scriptlist):
#             scriptname = 'observerscript_' + str(index)
#             scripts[scriptname] = script

#         # Making stamps and stuff

#         _model_inputs, _model_stamps, _model_hashID, _model_scripts = \
#             _scripts_and_stamps(system, initials)

#         stamps = {}
#         if uw.mpi.rank == 0:
#             stamps = {
#                 'inputs': hashstamp(obsInp),
#                 'scripts': hashstamp(
#                     [open(script) for scriptname, script \
#                         in sorted(scripts.items())]
#                     )
#                 }
#             for stampKey, stampVal in stamps.items():
#                 stamps[stampKey] = [stampVal, wordhashFn(stampVal)]
#             stamps['model_stamps'] = [_model_hashID, _model_stamps]
#         stamps = uw.mpi.comm.bcast(stamps, root = 0)

#         scripts.update(_model_scripts)
#         inputs = {
#             'obsInp': obsInp,
#             **_model_inputs,
#             }

#         # Making the checkpointer:

#         checkpointer = Checkpointer(
#             stamps = stamps,
#             step = system.step,
#             modeltime = system.modeltime,
#             figs = self.figs,
#             dataCollectors = self.collectors,
#             scripts = scripts,
#             inputs = inputs
#             )

#         self.checkpointer = checkpointer
#         self.inputs = inputs
#         self.scripts = scripts
#         self.stamps = stamps
#         self.system = system
#         self.initials = initials
#         self.step = system.step
#         self.modeltime = system.modeltime

#     def checkpoint(self, path):
#         self.checkpointer.checkpoint(path)

#     def prompt(self):
#         pass

In [3]:
import planetengine
system = planetengine.systems.arrhenius.build(res = 16)
initials = {'temperatureField': planetengine.initials.sinusoidal.IC(freq = 1.)}
planetengine.initials.apply(
    initials,
    system,
    )
system.solve()

In [4]:
observer = planetengine.observer.Observer(system, initials, ['./testfile.py'])

Making stamps...
Stamps made.


In [6]:
path = os.path.join(outputPath, 'obstest')
observer.checkpoint(path)

Pre-existing directory for this model has been found. Continuing...
Checkpointing...
Making checkpoint directory.
Saving figures...
Figures saved.
Saving vars of state...
Saved.
Saving snapshot...
Snapshot saved.
Saving modeltime...
Modeltime saved.
Saving stamps...
Stamps saved.
Saving datasets...
Datasets saved.
Checkpointed!


In [ ]:
system.iterate()

In [ ]:
observer.checkpoint(path)